In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from nltk.corpus import stopwords
import re
from tqdm import tqdm
import pickle

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
SPEED_RUN = True
tqdm.pandas(desc='')
DIR = './data/medium_articles/'

In [5]:
main_dataframe = pd.read_csv(DIR + 'medium_articles.csv')
if SPEED_RUN: text_dataframe = pd.read_csv(DIR + 'text_cleaned.csv')

main_dataframe.head(2)

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."


In [6]:
def process_tag(raw_tags: str):
    raw_tags = raw_tags.replace('\'', '').strip('[]').split(',')
    for i in range(len(raw_tags)):
        raw_tags[i] = raw_tags[i].strip()
    return raw_tags

main_dataframe['tags'] = main_dataframe['tags'].progress_apply(process_tag)

100%|██████████| 192368/192368 [00:00<00:00, 368810.08it/s]


In [7]:
def clean_text(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    eng_stopwords = set(stopwords.words('english'))
    text = text.lower()
    
    text = re.sub('\'', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)

    text = text.split()
    text = [stemmer.stem(lemmatizer.lemmatize(word)) for word in text]
    text = [word for word in text if word not in eng_stopwords]

    text = ' '.join(text).split('\n')

    return ' '.join(text)

In [8]:
if SPEED_RUN: 
    x = text_dataframe['text']
    del text_dataframe
else: x = main_dataframe['text'].progress_apply(clean_text)

x = x[x.isna() == False]

In [9]:
y = main_dataframe['tags'].loc[x.index]

tags_dict = {}
tag_occurence_treshold = 200
for tag_list in y:
    for tag in tag_list:
        if tag not in tags_dict:
            tags_dict[tag] = 1
        else:
            tags_dict[tag] += 1

def remove_rare_tags(tags: list, tags_list):
    tags = [tag for tag in tags if tag in tags_list]
    return tags

tags = [key for key, value in tags_dict.items() if value > tag_occurence_treshold]
y = y.progress_apply(lambda x: remove_rare_tags(x, tags))
y = y[y.str.len() > 0]

x = x.loc[y.index]

100%|██████████| 192367/192367 [00:05<00:00, 37934.53it/s]


In [10]:
x.shape, y.shape

((186665,), (186665,))

In [11]:
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10_000)
x_train_vectorized = tfidf_vectorizer.fit_transform(x_train)
x_test_vectorized = tfidf_vectorizer.transform(x_test)

In [13]:
if SPEED_RUN:
    with open(DIR + 'classifier.pkl', 'rb') as f:
        classifier = pickle.load(f)
else: 
    classifier = OneVsRestClassifier(LogisticRegression(max_iter=3000))
    classifier.fit(x_train_vectorized, y_train)

    with open(DIR + 'classifier.pkl', 'wb') as f:
        pickle.dump(classifier, f)

/Users/yourlogarithm/.miniforge3/envs/ml/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/yourlogarithm/.miniforge3/envs/ml/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/yourlogarithm/.miniforge3/envs/ml/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 1.0.2 when using 

<a href="https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter">max_iter=3000</a>

In [14]:
print(f'Train score: {classifier.score(x_train_vectorized, y_train): .2%}')
print(f'Test score: {classifier.score(x_test_vectorized, y_test): .2%}')

Train score:  3.52%
Test score:  2.84%


In [15]:
predictions = classifier.predict(x_test_vectorized)
f1_score(y_test, predictions, average='micro'), accuracy_score(y_test, predictions)

(0.2521894755466033, 0.028393110652773685)

In [16]:
def predict(text):
    text = clean_text(text)
    text = tfidf_vectorizer.transform([text])
    pred = classifier.predict(text)
    return list(mlb.inverse_transform(pred)[0])

In [18]:
for i in range(100):
    k = x_test.sample(1).index[0]
    print("Article: " + main_dataframe['title'][k])
    print("Predicted Tags: " + str(predict(x_test[k])))
    print("Actual Tags: " + str(main_dataframe['tags'][k]))
    print()

Article: Is Meditation an Escape or Fundamental to Change?
Predicted Tags: ['Meditation', 'Mindfulness']
Actual Tags: ['Indigeneity', 'Meditation', 'Activism', 'Sustainability', 'Mindfulness']

Article: TO REACH A MILLION, START WITH A PENNI
Predicted Tags: []
Actual Tags: ['Women Empowerment', 'Investing', 'Women', 'Real Estate', 'Financial Freedom']

Article: Points to consider while choosing a Professional Medical Billing Company
Predicted Tags: []
Actual Tags: ['Coding', 'Medical Billing', 'Healthcare', 'Medical', 'Finance']

Article: Album Review: ‘Phantoms’ by Marianas Trench
Predicted Tags: ['Album Review', 'Music', 'Rock']
Actual Tags: ['Pop Rock', 'Marianas Trench', 'Album Review', 'Review']

Article: 👀Huobi x CoinMarketCal👀A Coin To Keep An Eye On!
Predicted Tags: ['Blockchain']
Actual Tags: ['Huobi', 'Buyatom', 'Coinmarketcal', 'Cryptocurrency', 'Cosmos']

Article: Texts, Fonts, and Annotations with Python’s Matplotlib
Predicted Tags: ['Data Visualization']
Actual Tags: ['Te